# **Task11**

Use the code from: https://medium.com/@rekalantar/pytorch-tutorial-dynamic-weight-pruning-for-more-optimized-and-faster-neural-networks-7b337e47987b to do dynamic weight pruning. If the full dataset is too large use a suitable subset size.

Use post-training static quantization as well as dynamic quantization as given in: https://pytorch.org/tutorials/recipes/quantization.html to generate a dynamic weight pruning+quantized model. Compare accuracy of the quantized model vs the non-quantized model.


## Importing the required Libraries

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchsummary
from torchvision import models, datasets
from timeit import default_timer as timer
import torch.quantization as quant
import os

Checking for device

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

Loading the CIFAR10 dataset

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
# Load and subset the dataset
full_dataset = datasets.CIFAR10('./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(full_dataset, batch_size=100, shuffle=True)
# Load the test dataset
test_dataset = datasets.CIFAR10('./data', train=False, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

Files already downloaded and verified


# Creating a Model

In [4]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, input_channels, out_planes, stride= 1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, out_planes, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 =  nn.Conv2d(out_planes, out_planes, kernel_size=3,padding=1)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.downsample = downsample
        # FloatFunction()
        self.skip_add = nn.quantized.FloatFunctional()
        

    def forward(self, x):
        identity = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        if self.downsample is not None:
            identity = self.downsample(identity)
        # Notice the addition operation in both scenarios
        x  = self.skip_add.add(x, identity)
        x = self.relu(x)
        
        return x


class ResNet(nn.Module):
    
    def __init__(self, block=BasicBlock, layers=[2, 2, 2, 2], num_channels =3 ,num_classes=10):
        super(ResNet, self).__init__()
        self.input_channels = 64
        self.conv1 = nn.Conv2d(num_channels, self.input_channels, kernel_size = 7, stride = 2, padding = 3)
        self.bn1 = nn.BatchNorm2d(self.input_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        
        self.layer1 = self._make_layer(block, layers[0], out=64,  stride= 1)
        self.layer2 = self._make_layer(block, layers[1], out=128, stride= 2)
        self.layer3 = self._make_layer(block, layers[2], out=256, stride= 2)
        self.layer4 = self._make_layer(block, layers[3], out=512, stride= 2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()

    def _make_layer(self, block, residual_blocks ,out, stride):
        downsample = None
        layers = []
        if stride != 1 or self.input_channels != out:
            downsample = nn.Sequential(nn.Conv2d(self.input_channels, out, kernel_size=1, stride= stride),
                                       nn.BatchNorm2d(out))
        layers.append(block(self.input_channels, out, stride, downsample))
        self.input_channels = out
        for _ in range(1, residual_blocks):
            layers.append(block(self.input_channels, out))

        return nn.Sequential(*layers)

    
    def forward(self, x):
        # Input are quantized
        x = self.quant(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = self.dequant(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x

Calling the model ResNet()

In [5]:
model = ResNet()
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

### Defining the optimizer and loss function

In [6]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

### Function to count the zero parameters in the model

In [7]:
def count_zero_params(model):
    zero_count = 0
    for param in model.parameters():
        zero_count += torch.sum(param == 0).item()
    return zero_count

Printing the No.of Zero parameters

In [8]:
before_pruning = count_zero_params(model)
print(before_pruning)

4802


Printing the weights of the model

In [9]:
print("Original weights:\n", model.conv1.weight)

Original weights:
 Parameter containing:
tensor([[[[-0.0809, -0.0459,  0.0629,  ..., -0.0664,  0.0591,  0.0033],
          [ 0.0291, -0.0694, -0.0054,  ..., -0.0180, -0.0595,  0.0113],
          [ 0.0279,  0.0779, -0.0124,  ...,  0.0150,  0.0679,  0.0358],
          ...,
          [-0.0058, -0.0513,  0.0510,  ...,  0.0654,  0.0259,  0.0118],
          [ 0.0553, -0.0441, -0.0578,  ..., -0.0038,  0.0201,  0.0794],
          [ 0.0439,  0.0292,  0.0722,  ..., -0.0177, -0.0498,  0.0376]],

         [[-0.0271,  0.0134,  0.0677,  ..., -0.0722,  0.0160,  0.0775],
          [ 0.0193, -0.0159,  0.0811,  ...,  0.0671,  0.0367, -0.0198],
          [ 0.0666, -0.0800,  0.0025,  ..., -0.0101, -0.0251,  0.0305],
          ...,
          [-0.0534,  0.0653, -0.0506,  ..., -0.0572, -0.0387,  0.0645],
          [ 0.0333, -0.0278,  0.0549,  ..., -0.0267,  0.0116, -0.0498],
          [ 0.0232,  0.0253, -0.0682,  ..., -0.0501,  0.0749,  0.0295]],

         [[-0.0291,  0.0401,  0.0244,  ...,  0.0327,  0.0024,

Function to train the model and to do weight pruning

In [10]:
def train_pruned_model(model, device, train_loader, optimizer, criterion,epochs,prune_epoch):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch [{epoch + 1}/{epochs}], Loss : {running_loss / len(train_loader)}')       
        
        if (epoch+1) % prune_epoch == 0:
            prune_model(model, pruning_rate=0.1)
            print('Pruning done.')

In [11]:
import torch.nn.utils.prune as prune

def prune_model(model, pruning_rate=0.1):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            
            # Applying unstructured L1 norm pruning
            prune.l1_unstructured(module, name='weight', amount=pruning_rate)
            
            prune.remove(module, 'weight')

Function to Evaluate the model

In [12]:
import time
def evaluate_model(model,device):
    model = model.to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            st = time.time()
            outputs = model(images)
            et = time.time()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct // total} %')
    print('Elapsed time = {:0.4f} milliseconds'.format((et - st) * 1000))
    print("====================================================================================================")

## Calling the function train_model to train 

In [13]:
train_pruned_model(model, device, train_loader, optimizer,criterion, epochs=15,prune_epoch=5)

Epoch [1/15], Loss : 1.464614530801773
Epoch [2/15], Loss : 1.0113206499814986
Epoch [3/15], Loss : 0.807844424366951
Epoch [4/15], Loss : 0.6764954803586006
Epoch [5/15], Loss : 0.5684108149409294
Pruning done.
Epoch [6/15], Loss : 0.47543926364183425
Epoch [7/15], Loss : 0.39938797265291215
Epoch [8/15], Loss : 0.32666478598117826
Epoch [9/15], Loss : 0.2718150601387024
Epoch [10/15], Loss : 0.2161609608307481
Pruning done.
Epoch [11/15], Loss : 0.18002749675512314
Epoch [12/15], Loss : 0.1458682178556919
Epoch [13/15], Loss : 0.12177217001840472
Epoch [14/15], Loss : 0.09855391100049019
Epoch [15/15], Loss : 0.09286003717593849
Pruning done.


Printing the weights after weight pruning

In [17]:
print("Pruned weights:\n", model.conv1.weight)

Pruned weights:
 Parameter containing:
tensor([[[[-0.1515, -0.0626,  0.1119,  ..., -0.0182,  0.1126,  0.0513],
          [-0.0303, -0.1397,  0.0134,  ...,  0.0716,  0.0732,  0.0804],
          [-0.0718, -0.0964, -0.2018,  ..., -0.1349, -0.0255, -0.0354],
          ...,
          [ 0.1216,  0.1531,  0.1881,  ...,  0.0800,  0.0274, -0.0735],
          [ 0.0621,  0.0576,  0.0712,  ...,  0.0634,  0.0730,  0.0947],
          [ 0.0606,  0.0640,  0.1394,  ...,  0.0561,  0.0151,  0.0721]],

         [[-0.0850, -0.0264,  0.1218,  ..., -0.0149,  0.0655,  0.1063],
          [-0.0143, -0.0775,  0.1323,  ...,  0.2012,  0.1721,  0.0307],
          [-0.0431, -0.2524, -0.1634,  ..., -0.1150, -0.0809, -0.0000],
          ...,
          [ 0.0000,  0.1793,  0.0341,  ..., -0.0314, -0.0000,  0.0000],
          [ 0.0000,  0.0170,  0.1551,  ...,  0.0482,  0.0787, -0.0219],
          [ 0.0211,  0.0000, -0.0432,  ..., -0.0000,  0.1296,  0.0430]],

         [[-0.0000,  0.0471,  0.0531,  ..., -0.0000, -0.0267, -

No.of Zero parameters after weight pruning

In [18]:
after_pruning = count_zero_params(model)
print(after_pruning)

1117203


Evaluating the model and printing the accuracy

In [19]:
device ='cpu'
print("=====================================AFTER WEIGHT PRUNING========================================")
evaluate_model(model,device)

=====================================AFTER WEIGHT PRUNING========================================
Accuracy: 75 %
Elapsed time = 33.2420 milliseconds


Function to print the size of the model

In [20]:
def print_size_of_model(model,path):
    torch.save(model.state_dict(), path)
    print('Size (MB):', os.path.getsize(path)/1e3)

Printing the size of the model before quantization 

In [21]:
print("=====================================BEFORE QUANTIZATION========================================")
print(model)
print('Size of the model Before Dynamic quantization')
print_size_of_model(model,path="FinalTask11.pt")

=====================================BEFORE QUANTIZATION========================================
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), paddin

# Dynamic quantization

Doing dynamic quantization for the model

In [22]:
model_dynamic_quantized = torch.quantization.quantize_dynamic(
    model, qconfig_spec={torch.nn.Linear}, dtype=torch.qint8
)

Printing the model and size of it after dynamic quantization

In [23]:
print("=====================================DYNAMIC QUANTIZATION========================================")
print(model_dynamic_quantized)
print('Size of the model After Dynamic quantization')
print_size_of_model(model_dynamic_quantized,path="FinalTask11_dynamic.pt")

=====================================DYNAMIC QUANTIZATION========================================
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), paddi

Evaluating and printing the accuracy of the model after dynamic quantization

In [24]:
device ='cpu'
print("=====================================ACCURACY AFTER DYNAMIC QUANTIZATION=======================================")
evaluate_model(model_dynamic_quantized,device)

=====================================ACCURACY AFTER DYNAMIC QUANTIZATION=======================================
Accuracy: 76 %
Elapsed time = 27.3499 milliseconds


# Static Quantization

Doing Static quantization for the model

In [25]:
model_dynamic_quantized.eval()
model_dynamic_quantized.qconfig = torch.quantization.get_default_qconfig('fbgemm')
static_quantized = torch.quantization.prepare(model_dynamic_quantized, inplace=True)
print(static_quantized)
with torch.no_grad():
    for data, target in train_loader:
        static_quantized(data)
static_quantized = torch.quantization.convert(static_quantized, inplace=True)
print(static_quantized)

ResNet(
  (conv1): Conv2d(
    3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3)
    (activation_post_process): HistogramObserver()
  )
  (bn1): BatchNorm2d(
    64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (activation_post_process): HistogramObserver()
  )
  (relu): ReLU(
    inplace=True
    (activation_post_process): HistogramObserver()
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
        (activation_post_process): HistogramObserver()
      )
      (bn1): BatchNorm2d(
        64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (activation_post_process): HistogramObserver()
      )
      (relu): ReLU(
        inplace=True
        (activation_post_process): HistogramObserver()
      )
      (conv2): Conv2d(
        64, 64, kernel_size=(3, 3), stride=(1

ResNet(
  (conv1): QuantizedConv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.08372426778078079, zero_point=64, padding=(3, 3))
  (bn1): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): QuantizedReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.24363915622234344, zero_point=73, padding=(1, 1))
      (bn1): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): QuantizedReLU(inplace=True)
      (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.1066269651055336, zero_point=71, padding=(1, 1))
      (bn2): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): QFunctional(
        scale=0.16372664272785187, zero_point=36
       

Printing the model and size of it after Static quantization

In [26]:
print('Size of the model After Static quantization')
print_size_of_model(static_quantized,path="FinalTask11_static.pt")

Size of the model After Static quantization
Size (MB): 11420.771


Evaluating and finding the accuracy of the model 

In [27]:
device ='cpu'
print("=====================================ACCURACY AFTER STATIC QUANTIZATION=======================================")
evaluate_model(static_quantized,device)

=====================================ACCURACY AFTER STATIC QUANTIZATION=======================================
Accuracy: 76 %
Elapsed time = 10.3257 milliseconds
